In [2]:
import cv2
import os
import scanpy as sc
import numpy as np
import tifffile as tiff
import matplotlib.pyplot as plt
from PIL import Image
from PIL import ImageOps
import pandas as pd
from scipy.signal import savgol_filter
from collections import Counter

In [ ]:
#特别，yellow是会把图片黑色变为0，clipLimit，clipLimit，通道1
# blue是会把图片黑色变为clipLimit，0，0，通道0
# red是会把图片黑色变为0，0，clipLimit，通道2
# green是会把图片黑色变为0，clipLimit，0，通道1
# white是会把图片黑色变为clipLimit，clipLimit，clipLimit，通道0

def tiff_channel_to_png_background(panel, channel, color, image):
    panel = panel.query("marker == @channel")
    panel = panel.reset_index().rename(columns={'index': 'Channel_index'})
    result = [
        panel[panel['marker'] == channel]['Channel_index'].iloc[0],
    ]
    image_channel = image[result[0],:,:] 
    image_channel = (image_channel - np.min(image_channel)) / (np.max(image_channel) - np.min(image_channel))# 对图像进行归一化
    # 将归一化后的图像数据类型转换为整型
    image_channel = (image_channel * 255).astype(np.uint8)
    # 对图像进行直方图均衡化
    # titleGridSize进行像素均衡化的网格大小，即在多少网格下进行直方图的均衡化操作
    # clipLimit颜色对比度的阈值
    clahe2 = cv2.createCLAHE(clipLimit=10, tileGridSize=(50, 50))
    image_channel = clahe2.apply(image_channel)
    
    image_channel = Image.fromarray(image_channel)
    image_channel = ImageOps.colorize(image_channel, "#000000", white = color)
    # 将PIL图像转换为OpenCV图像
    img = cv2.cvtColor(np.array(image_channel), cv2.COLOR_RGB2BGR)
    return img

def tiff_channel_to_png_mask(panel, channel, color, image, mask):
    panel = panel.query("marker == @channel")
    panel = panel.reset_index().rename(columns={'index': 'Channel_index'})
    result = [
        panel[panel['marker'] == channel]['Channel_index'].iloc[0],
    ]
    image_channel = image[result[0],:,:] 
    image_channel = (image_channel - np.min(image_channel)) / (np.max(image_channel) - np.min(image_channel))# 对图像进行归一化
    image_channel = (image_channel * 255).astype(np.uint8)
    clahe2 = cv2.createCLAHE(clipLimit=10, tileGridSize=(50, 50))
    image_channel = clahe2.apply(image_channel)
    image_channel = Image.fromarray(image_channel)
    image_channel = ImageOps.colorize(image_channel, "#000000", white = color)
    background = cv2.cvtColor(np.array(image_channel), cv2.COLOR_RGB2BGR)
    
    
    # mask处理------------------------------------------------------------------------------------------
    mask = mask[0:background.shape[0],0:background.shape[1]]
    mask_normalized = (mask - np.min(mask)) / (np.max(mask) - np.min(mask))
    mask_normalized = (mask_normalized * 255).astype(np.uint8)

    clahe2 = cv2.createCLAHE(clipLimit=20, tileGridSize=(150, 150))
    mask_eq = clahe2.apply(mask_normalized)
    
    #提取mask图像边缘轮廓
    edges = cv2.Canny(mask_eq, 2, 50)
    edges = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)

    # 创建一个具有alpha通道的空白图像，大小与灰度图像edges相同
    height, width,_ = edges.shape
    rgba_edges = np.zeros((height, width, 4), dtype=np.uint8)

    # 将灰度图像edges复制到alpha通道中
    rgba_edges[:, :, 0:3] = edges
    # 将灰度值为0的像素设置为完全透明，将灰度值为255的像素设置为不透明
    rgba_edges[rgba_edges[:, :, 2] == 0] = [0, 0, 0, 0]
    rgba_edges[rgba_edges[:, :, 2] == 255] = [100, 100, 100, 255]

    # 创建一个空白的图像，大小与三通道图像img1相同
    img = np.zeros(background.shape, dtype=np.uint8)
    # 在结果图像上，将完全透明的像素设置为三通道图像background的对应像素
    img[rgba_edges[:, :, 3] == 0] = background[rgba_edges[:, :, 3] == 0]
    # 在结果图像上，将不透明的像素设置为灰度图像A的对应像素
    img[rgba_edges[:,:,3]==255] = rgba_edges[rgba_edges[:,:,3]==255,:3]

    return img

def tiff_channel_to_png(panel, channel, color, image, background):        
    panel = panel.query("marker == @channel")
    panel = panel.reset_index().rename(columns={'index': 'Channel_index'})
    result = [
        panel[panel['marker'] == channel]['Channel_index'].iloc[0],
    ]
    image_channel = image[result[0],:,:] 
    image_channel = (image_channel - np.min(image_channel)) / (np.max(image_channel) - np.min(image_channel))# 对图像进行归一化
    image_channel = (image_channel * 255).astype(np.uint8)

    clahe2 = cv2.createCLAHE(clipLimit=10, tileGridSize=(50, 50))
    image_channel = clahe2.apply(image_channel)
    
    image_channel = Image.fromarray(image_channel)
    image_channel = ImageOps.colorize(image_channel, "#000000", white = color)
    image_channel = cv2.cvtColor(np.array(image_channel), cv2.COLOR_RGB2BGR)
    height, width, _ = image_channel.shape
    rgba = np.zeros((height, width, 4), dtype=np.uint8)
#     # 遍历图像像素点
#     for i in range(300):
#         for j in range(image_channel.shape[1]):
#             print('Pixel Value at ({}, {}):'.format(i, j), image_channel[i, j])

    if color == 'blue' or color == 'white':
        i = 0
    elif color == 'yellow' or color == 'green':
        i = 1
    elif color == 'red':
        i = 2       
    condition1 = image_channel[:,:,i] >=11
    rgba[:, :, 0:3] = image_channel
    rgba[:, :, 3] = np.where(condition1,255,0)

    img = np.zeros(background.shape, dtype=np.uint8)
    img[rgba[:, :, 3] == 0] = background[rgba[:, :, 3] == 0]
    img[rgba[:,:,3]==255] = rgba[rgba[:,:,3]==255,:3]   
    return img

In [2]:
# 2.读取Tiff格式图像
panel_file = "/mnt/data/lyx/IMC/IMC_CRC_panel_v2.csv" 
multi_tiff_file = '/mnt/data/lyx/IMC/fullstacks/20221208_B13_ROI7_fullstacks.tiff'
mask_file = '/mnt/data/lyx/IMC/unet/predictionProbability/ori_cellmask/20221208_B13_ROI8_pred_Probabilities_cell_mask.tiff'
image = tiff.imread(multi_tiff_file)
mask = tiff.imread(mask_file)
panel = pd.read_csv(panel_file)

In [11]:
# 通道39：DNA2
# 通道22：161Dy_CD20
# 通道31：170Er_CD3

color1 = "blue"
color2= "white"
color3= "red"
color4= "green"
color5= "yellow"

channel_1 = "DNA2"
channel_2  = "FoxP3"
channel_3 = "CD279"
channel_4 = "TIGIT"
channel_5 = "CD57"

result_img1 =  tiff_channel_to_png_background(panel,channel_1,color1,image)
cv2.imwrite('./result_image1.png', result_img1)
result_img2=tiff_channel_to_png(panel, channel_2, color2, image, result_img1)
cv2.imwrite('./result_image2.png', result_img2)
result_img3=tiff_channel_to_png(panel, channel_3, color3, image, result_img2)
cv2.imwrite('./result_image3.png', result_img3)
result_img4=tiff_channel_to_png(panel, channel_4, color4, image, result_img3)
cv2.imwrite('./result_image4.png', result_img4)
result_img5=tiff_channel_to_png(panel, channel_5, color5, image, result_img4)
cv2.imwrite('./result_image5.png', result_img5)

True

In [9]:
## plot cell mask burden
mask_img = tiff_channel_to_png_mask(panel, channel_2, color2, image, mask)
cv2.imwrite('./mask_img_'+channel_2+'.png', mask_img)
mask_img = tiff_channel_to_png_mask(panel, channel_3, color3, image, mask)
cv2.imwrite('./mask_img_'+channel_3+'.png', mask_img)

True

## plot cell type on mask

In [3]:
def get_color_for_subtype(subtype):
    # Mapping of subtypes to colors (RGBA). Adjust colors as needed.
    color_map = {
        'SC_aSMA': [255, 0, 0, 255],
        'Macro_HLADR': [0, 255, 0, 255],
        'Mono_CD11c': [0, 0, 255, 255],
        'CD4T': [255, 255, 0, 255], # yellow
        'SC_Vimentin': [0, 255, 255, 255], # cyan
        'SC_COLLAGEN': [255, 0, 255, 255], # magenta
        'TC_EpCAM': [128, 0, 0, 255], # maroon
        'B': [0, 128, 0, 255], # green
        'TC_VEGF': [0, 0, 128, 255], # navy
        'SC_FAP': [128, 128, 0, 255], # olive
        'Macro_CD169': [128, 0, 128, 255], # purple
        'UNKNOWN': [0, 128, 128, 255], # teal
        'TC_Ki67': [192, 192, 192, 255], # silver
        'Macro_CD163': [128, 128, 128, 255], # gray
        'Macro_CD11b': [255, 165, 0, 255], # orange
        'CD8T': [0, 255, 127, 255], # spring green
        'NK': [220, 20, 60, 255], # crimson
        'Mono_Intermediate': [255, 0, 127, 255], # pink
        'Mono_Classic': [75, 0, 130, 255], # indigo
        'Treg': [0, 191, 255, 255], # deep sky blue
        'TC_CAIX': [85, 107, 47, 255] # dark olive green
    }
    
    return color_map.get(subtype, [255, 255, 255, 255])  # Default to white if subtype not found


In [18]:
def generate_cell_mask(image_shape, mask, df, savepath):
    mask = mask[0:image_shape[0], 0:image_shape[1]]
    mask_normalized = (mask - np.min(mask)) / (np.max(mask) - np.min(mask))
    mask_normalized = (mask_normalized * 255).astype(np.uint8)

    clahe2 = cv2.createCLAHE(clipLimit=20, tileGridSize=(150, 150))
    mask_eq = clahe2.apply(mask_normalized)

    # Extract mask image edges
    edges = cv2.Canny(mask_eq, 2, 50)
    edges = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)

    # Create an empty image with alpha channel, the same size as the grayscale image edges
    height, width, _ = edges.shape
    rgba_edges = np.zeros((height, width, 4), dtype=np.uint8)

    # Copy the grayscale image edges into the alpha channel
    rgba_edges[:, :, 0:3] = edges
    # Set pixels with grayscale value of 0 to be fully transparent, and pixels with grayscale value of 255 to be opaque
    rgba_edges[rgba_edges[:, :, 2] == 0] = [0, 0, 0, 0]
    rgba_edges[rgba_edges[:, :, 2] == 255] = [100, 100, 100, 255]

    for index, row in df.iterrows():
        # Mark each cell's position based on the df
        x, y = int(row['x']), int(row['y'])
        subtype_color = get_color_for_subtype(row['SubType'])
        cv2.circle(rgba_edges, (x, y), radius=2, color=subtype_color, thickness=-1)
        
    # To save the image to a file
    cv2.imwrite(savepath, rgba_edges)

    return None


In [19]:
def generate_cell_mask2(image_shape, mask, df, savepath):
    mask = mask[0:image_shape[0], 0:image_shape[1]]
    mask_normalized = (mask - np.min(mask)) / (np.max(mask) - np.min(mask))
    mask_normalized = (mask_normalized * 255).astype(np.uint8)

    clahe2 = cv2.createCLAHE(clipLimit=20, tileGridSize=(150, 150))
    mask_eq = clahe2.apply(mask_normalized)

    # Extract mask image edges
    edges = cv2.Canny(mask_eq, 2, 50)
    edges = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)

    # Create an empty image with alpha channel, the same size as the grayscale image edges
    height, width, _ = edges.shape
    rgba_edges = np.zeros((height, width, 4), dtype=np.uint8)

    # Copy the grayscale image edges into the alpha channel
    rgba_edges[:, :, 0:3] = edges
    # Set pixels with grayscale value of 0 to be fully transparent, and pixels with grayscale value of 255 to be opaque
    rgba_edges[rgba_edges[:, :, 2] == 0] = [0, 0, 0, 0]
    rgba_edges[rgba_edges[:, :, 2] == 255] = [100, 100, 100, 255]

    # Separate the RGB channels for flood fill and keep a separate alpha channel
    rgb_edges = rgba_edges[:, :, 0:3].copy().astype(np.uint8)  # Ensure it's uint8 and create a copy
    alpha_channel = rgba_edges[:, :, 3]

    # Make a mask for flood fill, 2 pixels bigger than our image and filled with zeros
    h, w = rgb_edges.shape[:2]
    flood_mask = np.zeros((h+2, w+2), dtype=np.uint8)

    for index, row in df.iterrows():
        # Fill cell regions using flood fill
        x, y = int(row['x']), int(row['y'])
        subtype_color = get_color_for_subtype(row['SubType'])
        # The flood fill modifies the mask in-place, so we need to reset it each time
        flood_mask[:] = 0
        # Use flood fill starting from the cell's center. The edges will act as boundaries.
        cv2.floodFill(rgb_edges, flood_mask, (x, y), subtype_color)

    # Merge the RGB and alpha channel back to an RGBA image
    rgba_result = cv2.merge([rgb_edges[:, :, 0], rgb_edges[:, :, 1], rgb_edges[:, :, 2], alpha_channel])

    # To save the image to a file
    cv2.imwrite(savepath, rgba_result)

    return None

In [11]:
adata = sc.read_h5ad("/mnt/data/lyx/IMC/analysis/spatial/adata.h5ad")
all_mask_path = "/mnt/data/lyx/IMC/unet/predictionProbability/ori_cellmask"
all_mask_files = os.listdir(all_mask_path)

save_dir = "/mnt/data/lyx/IMC/analysis/spatial/celltype_with_mask"
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

In [20]:
ROIs = [x for x in Counter(adata.obs["ID"]).keys()]

for ROI in ROIs:
    adata_ = adata[adata.obs['ID'].isin([ROI]),]
    df = adata_.obs.loc[:,["SubType","x","y"]]

    # Load
    mask_file = [filename for filename in all_mask_files if ROI in filename][0]
    image_shape = [1000,1000]
    mask = tiff.imread(os.path.join(all_mask_path,mask_file))

    # Plot
    output_img = generate_cell_mask(image_shape, mask, df, savepath = os.path.join(save_dir,ROI+"_type_with_mask.tif"))

In [23]:
import cv2
from PIL import Image

# Convert the saved image to a PDF using Pillow
image = Image.open('/mnt/data/lyx/IMC/analysis/spatial/celltype_with_mask/B10_ROI11_type_with_mask.tif')

# Convert the image from RGBA to RGB
image_rgb = image.convert("RGB")

image_rgb.save('/mnt/data/lyx/IMC/analysis/spatial/celltype_with_mask/B10_ROI11_type_with_mask.pdf', "PDF", resolution=100.0)